In [ ]:
#!/usr/bin/env python3
"""
Complete ARC Solver with Neural Networks and Rule-Based Patterns
Optimized for Google Code Golf Championship 2025
"""

import os
import sys
import json
import zipfile
import numpy as np
from collections import defaultdict, Counter
import traceback
import warnings
warnings.filterwarnings('ignore')

# Install required packages if in Kaggle environment
if '/kaggle' in sys.path[0]:
    print("🔧 Installing required packages...")
    os.system('pip install -q torch torchvision')
    print("✅ Packages installed\n")

# Import optional packages
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    TORCH_AVAILABLE = True
except:
    TORCH_AVAILABLE = False
    print("⚠️  PyTorch not available, using rule-based patterns only\n")

class SimpleARCNet(nn.Module):
    """Simple neural network for ARC pattern recognition"""
    def __init__(self, max_size=30):
        super(SimpleARCNet, self).__init__()
        self.max_size = max_size
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * max_size * max_size, 256)
        self.fc2 = nn.Linear(256, 100)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class ComprehensiveARCSolver:
    def __init__(self, enable_neural=True, verbose=True):
        self.enable_neural = enable_neural and TORCH_AVAILABLE
        self.verbose = verbose
        self.solutions_cache = {}
        self.pattern_stats = defaultdict(lambda: {'attempts': 0, 'successes': 0})
        
        if self.enable_neural:
            self.net = SimpleARCNet()
            if self.verbose:
                print("🧠 Neural network initialized")
        
        # Comprehensive pattern library with golf-optimized implementations
        self.patterns = {
            # Basic transformations
            'identity': "def p(g):\n return g",
            'rotate_90': "def p(g):\n return[list(r)for r in zip(*g[::-1])]",
            'rotate_180': "def p(g):\n return[r[::-1]for r in g[::-1]]",
            'rotate_270': "def p(g):\n return[list(r)for r in zip(*g)][::-1]",
            'flip_h': "def p(g):\n return[r[::-1]for r in g]",
            'flip_v': "def p(g):\n return g[::-1]",
            'transpose': "def p(g):\n return[list(r)for r in zip(*g)]",
            
            # Scaling
            'scale_2x': "def p(g):\n return[[g[i//2][j//2]for j in range(len(g[0])*2)]for i in range(len(g)*2)]",
            'scale_3x': "def p(g):\n return[[g[i//3][j//3]for j in range(len(g[0])*3)]for i in range(len(g)*3)]",
            'downscale_2x': "def p(g):\n return[r[::2]for r in g[::2]]",
            
            # Color operations
            'fill_zeros': "def p(g):\n return[[0]*len(g[0])for _ in g]",
            'fill_ones': "def p(g):\n return[[1]*len(g[0])for _ in g]",
            'invert': "def p(g):\n return[[9-x for x in r]for r in g]",
            
            # Gravity
            'gravity_down': "def p(g):\n return[[0]*len(g[0])for _ in range(len(g)-sum(1 for r in g if any(r)))]+[r for r in g if any(r)]",
            'gravity_up': "def p(g):\n return[r for r in g if any(r)]+[[0]*len(g[0])for _ in range(len(g)-sum(1 for r in g if any(r)))]",
            'gravity_left': "def p(g):\n return[[x for x in r if x]+[0]*(len(r)-sum(1 for x in r if x))for r in g]",
            'gravity_right': "def p(g):\n return[[0]*(len(r)-sum(1 for x in r if x))+[x for x in r if x]for r in g]",
            
            # Compression
            'compress_v': "def p(g):\n return[r for r in g if any(r)]",
            'compress_h': "def p(g):\n return[list(r)for r in zip(*[c for c in zip(*g)if any(c)])]",
            
            # Object operations
            'outline': "def p(g):\n h,w=len(g),len(g[0]);return[[g[i][j]if g[i][j]and any((y<0 or y>=h or x<0 or x>=w or g[y][x]==0)for y,x in[(i-1,j),(i+1,j),(i,j-1),(i,j+1)])else 0 for j in range(w)]for i in range(h)]",
            
            # Row/column operations
            'fill_row_maj': "def p(g):\n return[[max(r,key=r.count)]*len(r)for r in g]",
            'sort_rows': "def p(g):\n return sorted(g,key=lambda r:sum(x>0 for x in r))",
            
            # Arithmetic
            'add_one': "def p(g):\n return[[x+1 for x in r]for r in g]",
            'mod_two': "def p(g):\n return[[x%2 for x in r]for r in g]",
            'double': "def p(g):\n return[[x*2 for x in r]for r in g]",
        }
        
        # Dynamic pattern handlers
        self.dynamic_handlers = [
            self.detect_color_mapping,
            self.detect_single_color_fill,
            self.detect_crop,
            self.detect_tile,
            self.detect_extract_color,
            self.detect_diagonal,
            self.detect_cross,
            self.detect_checkerboard,
            self.detect_frame,
            self.detect_bounding_box,
            self.detect_mirror,
            self.detect_unique_colors,
            self.detect_count_pattern,
            self.detect_subgrid,
            self.detect_stripes,
            self.detect_gradient,
            self.detect_row_repeat,
            self.detect_swap_colors,
            self.detect_arithmetic_pattern,
            self.detect_object_move,
            self.detect_fill_shape,
            self.detect_extract_largest,
            self.detect_connect_dots,
            self.detect_symmetry_complete,
            self.detect_pixel_art_scale,
        ]
    
    def solve_task(self, task_data, task_id="unknown"):
        """Main solving method with extensive diagnostics"""
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"🔍 Analyzing Task {task_id}")
            print(f"{'='*60}")
        
        # Cache check
        task_str = json.dumps(task_data, sort_keys=True)
        task_hash = hash(task_str)
        if task_hash in self.solutions_cache:
            if self.verbose:
                print("📦 Using cached solution")
            return self.solutions_cache[task_hash]
        
        # Analyze task
        analysis = self.analyze_task(task_data)
        if self.verbose:
            self.print_analysis(analysis)
        
        # Try static patterns
        if self.verbose:
            print("\n🔧 Trying static patterns...")
        
        for pattern_name, code in self.patterns.items():
            self.pattern_stats[pattern_name]['attempts'] += 1
            if self._verify_solution(code, task_data):
                self.pattern_stats[pattern_name]['successes'] += 1
                if self.verbose:
                    print(f"✅ Pattern '{pattern_name}' works!")
                self.solutions_cache[task_hash] = code
                return code
        
        # Try dynamic patterns
        if self.verbose:
            print("\n🔍 Trying dynamic patterns...")
        
        for handler in self.dynamic_handlers:
            handler_name = handler.__name__
            self.pattern_stats[handler_name]['attempts'] += 1
            try:
                code = handler(task_data, analysis)
                if code and self._verify_solution(code, task_data):
                    self.pattern_stats[handler_name]['successes'] += 1
                    if self.verbose:
                        print(f"✅ Dynamic pattern '{handler_name}' works!")
                    self.solutions_cache[task_hash] = code
                    return code
            except Exception as e:
                if self.verbose:
                    print(f"❌ Error in {handler_name}: {str(e)}")
        
        # Try neural network if available
        if self.enable_neural and self.verbose:
            print("\n🧠 Trying neural network approach...")
            # Neural network code would go here
            # For now, we skip as it requires training data
        
        # Try combined patterns
        if self.verbose:
            print("\n🔗 Trying combined patterns...")
        combined_code = self.try_combined_patterns(task_data, analysis)
        if combined_code:
            if self.verbose:
                print("✅ Combined pattern works!")
            self.solutions_cache[task_hash] = combined_code
            return combined_code
        
        # Fallback
        if self.verbose:
            print("\n⚠️  Using fallback (identity)")
        return "def p(g):\n return g"
    
    def analyze_task(self, task_data):
        """Comprehensive task analysis"""
        analysis = {
            'input_shapes': [],
            'output_shapes': [],
            'color_counts': defaultdict(int),
            'transformations': [],
            'shape_consistent': True,
            'color_mapping': {},
            'objects': [],
            'symmetries': [],
            'patterns': []
        }
        
        for example in task_data['train']:
            inp = np.array(example['input'])
            out = np.array(example['output'])
            
            analysis['input_shapes'].append(inp.shape)
            analysis['output_shapes'].append(out.shape)
            
            # Color analysis
            for color in np.unique(inp):
                analysis['color_counts'][int(color)] += 1
            
            # Shape consistency
            if inp.shape != out.shape:
                analysis['shape_consistent'] = False
            
            # Detect transformations
            if np.array_equal(out, np.rot90(inp)):
                analysis['transformations'].append('rotate_90')
            elif np.array_equal(out, inp[::-1]):
                analysis['transformations'].append('flip_v')
            elif np.array_equal(out, inp[:, ::-1]):
                analysis['transformations'].append('flip_h')
        
        return analysis
    
    def print_analysis(self, analysis):
        """Print detailed analysis"""
        print("\n📊 Task Analysis:")
        print(f"  Input shapes: {analysis['input_shapes']}")
        print(f"  Output shapes: {analysis['output_shapes']}")
        print(f"  Shape consistent: {analysis['shape_consistent']}")
        print(f"  Colors used: {dict(analysis['color_counts'])}")
        if analysis['transformations']:
            print(f"  Detected transformations: {analysis['transformations']}")
    
    def _verify_solution(self, code, task_data):
        """Verify if solution works for all examples"""
        try:
            # Create namespace and execute code
            namespace = {}
            exec(code, namespace)
            p = namespace['p']
            
            # Test on all training examples
            for i, example in enumerate(task_data['train']):
                input_grid = [row[:] for row in example['input']]
                expected = example['output']
                
                try:
                    result = p(input_grid)
                    if result != expected:
                        return False
                except:
                    return False
            
            return True
        except Exception as e:
            if self.verbose:
                print(f"    Verification error: {str(e)}")
            return False
    
    # Dynamic pattern detection methods
    def detect_color_mapping(self, task_data, analysis):
        """Detect and generate color mapping code"""
        color_map = {}
        
        for example in task_data['train']:
            inp = np.array(example['input'])
            out = np.array(example['output'])
            
            if inp.shape != out.shape:
                return None
            
            # Build color mapping
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    c_in = int(inp[i, j])
                    c_out = int(out[i, j])
                    if c_in in color_map and color_map[c_in] != c_out:
                        return None
                    color_map[c_in] = c_out
        
        if color_map and len(color_map) > 1:
            # Generate code with proper formatting
            map_items = [f"{k}:{v}" for k, v in sorted(color_map.items())]
            map_str = "{" + ",".join(map_items) + "}"
            return f"def p(g):\n m={map_str}\n return[[m.get(x,x)for x in r]for r in g]"
        
        return None
    
    def detect_single_color_fill(self, task_data, analysis):
        """Detect single color fill pattern"""
        colors = set()
        shapes = set()
        
        for example in task_data['train']:
            out = example['output']
            out_flat = [x for row in out for x in row]
            unique_colors = set(out_flat)
            
            if len(unique_colors) != 1:
                return None
            
            colors.update(unique_colors)
            shapes.add((len(out), len(out[0])))
        
        if len(colors) == 1 and len(shapes) == 1:
            color = colors.pop()
            h, w = shapes.pop()
            return f"def p(g):\n return[[{color}]*{w}for _ in range({h})]"
        
        return None
    
    def detect_crop(self, task_data, analysis):
        """Detect cropping pattern"""
        out_shapes = list(set(analysis['output_shapes']))
        if len(out_shapes) == 1 and all(s[0] > out_shapes[0][0] or s[1] > out_shapes[0][1] 
                                       for s in analysis['input_shapes']):
            h, w = out_shapes[0]
            return f"def p(g):\n return[r[:{w}]for r in g[:{h}]]"
        return None
    
    def detect_tile(self, task_data, analysis):
        """Detect tiling pattern"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
            ty = out.shape[0] // inp.shape[0]
            tx = out.shape[1] // inp.shape[1]
            
            # Verify tiling
            is_tiled = all(out[i, j] == inp[i % inp.shape[0], j % inp.shape[1]]
                          for i in range(out.shape[0])
                          for j in range(out.shape[1]))
            
            if is_tiled:
                h, w = out.shape
                return f"def p(g):\n return[[g[i%len(g)][j%len(g[0])]for j in range({w})]for i in range({h})]"
        
        return None
    
    def detect_extract_color(self, task_data, analysis):
        """Detect color extraction pattern"""
        for color in range(1, 10):
            valid = True
            for example in task_data['train']:
                out = np.array(example['output'])
                if not np.all((out == color) | (out == 0)):
                    valid = False
                    break
            
            if valid:
                return f"def p(g):\n return[[x if x=={color}else 0 for x in r]for r in g]"
        
        return None
    
    def detect_diagonal(self, task_data, analysis):
        """Detect diagonal line pattern"""
        ex = task_data['train'][0]
        out = np.array(ex['output'])
        
        if len(out) == len(out[0]):  # Square
            # Main diagonal
            if all(out[i, i] == out[0, 0] for i in range(len(out))):
                c = int(out[0, 0])
                return f"def p(g):\n n=len(g)\n return[[{c}if i==j else g[i][j]for j in range(n)]for i in range(n)]"
        
        return None
    
    def detect_cross(self, task_data, analysis):
        """Detect cross pattern"""
        ex = task_data['train'][0]
        out = np.array(ex['output'])
        h, w = out.shape
        cy, cx = h // 2, w // 2
        
        if len(set(out[cy, :])) == 1 and len(set(out[:, cx])) == 1:
            c = int(out[cy, cx])
            return f"def p(g):\n h,w=len(g),len(g[0]);cy,cx=h//2,w//2\n return[[{c}if i==cy or j==cx else g[i][j]for j in range(w)]for i in range(h)]"
        
        return None
    
    def detect_checkerboard(self, task_data, analysis):
        """Detect checkerboard pattern"""
        ex = task_data['train'][0]
        out = np.array(ex['output'])
        
        if len(np.unique(out)) == 2:
            c0, c1 = sorted(map(int, np.unique(out)))
            is_checker = all(out[i, j] == (c0 if (i + j) % 2 == 0 else c1)
                           for i in range(out.shape[0])
                           for j in range(out.shape[1]))
            
            if is_checker:
                h, w = out.shape
                return f"def p(g):\n return[[{c0}if(i+j)%2==0 else{c1}for j in range({w})]for i in range({h})]"
        
        return None
    
    def detect_frame(self, task_data, analysis):
        """Detect frame pattern"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        if out.shape == (inp.shape[0] + 2, inp.shape[1] + 2):
            if np.array_equal(out[1:-1, 1:-1], inp):
                c = int(out[0, 0])
                return f"def p(g):\n h,w=len(g),len(g[0]);f={c}\n return[[f]*(w+2)]+[[f]+r+[f]for r in g]+[[f]*(w+2)]"
        
        return None
    
    def detect_bounding_box(self, task_data, analysis):
        """Detect bounding box extraction"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        nonzero = np.argwhere(inp > 0)
        if len(nonzero) > 0:
            y_min, x_min = nonzero.min(axis=0)
            y_max, x_max = nonzero.max(axis=0)
            
            extracted = inp[y_min:y_max+1, x_min:x_max+1]
            if np.array_equal(extracted, out):
                return """def p(g):
 c=[(i,j)for i,r in enumerate(g)for j,x in enumerate(r)if x]
 if c:
  y0=min(i for i,j in c);y1=max(i for i,j in c)
  x0=min(j for i,j in c);x1=max(j for i,j in c)
  return[g[i][x0:x1+1]for i in range(y0,y1+1)]
 return g"""
        
        return None
    
    def detect_mirror(self, task_data, analysis):
        """Detect mirror patterns"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        if inp.shape == out.shape:
            h, w = inp.shape
            # Check horizontal mirror
            if w % 2 == 0 and np.array_equal(out[:, :w//2], out[:, w//2:][:, ::-1]):
                return "def p(g):\n return[r[:len(r)//2]+r[:len(r)//2][::-1]for r in g]"
            # Check vertical mirror
            if h % 2 == 0 and np.array_equal(out[:h//2, :], out[h//2:, :][::-1]):
                return "def p(g):\n return g[:len(g)//2]+g[:len(g)//2][::-1]"
        
        return None
    
    def detect_unique_colors(self, task_data, analysis):
        """Detect unique colors extraction"""
        ex = task_data['train'][0]
        if len(ex['output']) == 1:
            inp_colors = sorted(set(x for row in ex['input'] for x in row if x != 0))
            if ex['output'][0] == inp_colors:
                return "def p(g):\n return[sorted(set(x for r in g for x in r if x))]"
        return None
    
    def detect_count_pattern(self, task_data, analysis):
        """Detect counting patterns"""
        ex = task_data['train'][0]
        if all(len(row) == 1 for row in ex['output']):
            # Count non-zeros per row
            counts = [sum(1 for x in row if x != 0) for row in ex['input']]
            if [row[0] for row in ex['output']] == counts:
                return "def p(g):\n return[[sum(1 for x in r if x)]for r in g]"
        return None
    
    def detect_subgrid(self, task_data, analysis):
        """Detect subgrid extraction"""
        ex = task_data['train'][0]
        inp = ex['input']
        out = ex['output']
        
        if len(out) < len(inp) and len(out[0]) < len(inp[0]):
            h, w = len(out), len(out[0])
            # Try different positions
            for y in range(len(inp) - h + 1):
                for x in range(len(inp[0]) - w + 1):
                    subgrid = [inp[i][x:x+w] for i in range(y, y+h)]
                    if subgrid == out:
                        return f"def p(g):\n return[g[i][{x}:{x+w}]for i in range({y},{y+h})]"
        
        return None
    
    def detect_stripes(self, task_data, analysis):
        """Detect stripe patterns"""
        ex = task_data['train'][0]
        out = np.array(ex['output'])
        h, w = out.shape
        
        # Horizontal stripes
        if h >= 2:
            colors = []
            for i in range(min(2, h)):
                if len(set(out[i])) == 1:
                    colors.append(int(out[i, 0]))
            
            if len(colors) == 2:
                is_striped = all(len(set(out[i])) == 1 and int(out[i, 0]) == colors[i % 2] 
                               for i in range(h))
                if is_striped:
                    return f"def p(g):\n return[[{colors[0]}if i%2==0 else{colors[1]}for _ in range({w})]for i in range({h})]"
        
        return None
    
    def detect_gradient(self, task_data, analysis):
        """Detect gradient patterns"""
        ex = task_data['train'][0]
        out = np.array(ex['output'])
        h, w = out.shape
        
        # Horizontal gradient
        is_h_grad = all(len(set(out[:, j])) == 1 for j in range(w))
        if is_h_grad:
            values = [int(out[0, j]) for j in range(w)]
            if values == list(range(w)):
                return f"def p(g):\n return[[j for j in range({w})]for _ in range({h})]"
        
        # Vertical gradient
        is_v_grad = all(len(set(out[i, :])) == 1 for i in range(h))
        if is_v_grad:
            values = [int(out[i, 0]) for i in range(h)]
            if values == list(range(h)):
                return f"def p(g):\n return[[i]*{w}for i in range({h})]"
        
        # Diagonal gradient
        is_diag = all(out[i, j] == (i + j) % 10 for i in range(h) for j in range(w))
        if is_diag:
            return f"def p(g):\n return[[(i+j)%10 for j in range({w})]for i in range({h})]"
        
        return None
    
    def detect_row_repeat(self, task_data, analysis):
        """Detect row repetition"""
        ex = task_data['train'][0]
        if len(ex['output']) > len(ex['input']):
            for i, row in enumerate(ex['input']):
                if all(out_row == row for out_row in ex['output']):
                    return f"def p(g):\n return[g[{i}]]*{len(ex['output'])}"
        return None
    
    def detect_swap_colors(self, task_data, analysis):
        """Detect color swapping"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        if inp.shape == out.shape:
            inp_colors = set(inp.flatten())
            out_colors = set(out.flatten())
            
            if len(inp_colors) == 2 and inp_colors == out_colors:
                c1, c2 = sorted(inp_colors)
                if np.all((inp == c1) == (out == c2)) and np.all((inp == c2) == (out == c1)):
                    return f"def p(g):\n return[[{c2}if x=={c1}else{c1}if x=={c2}else x for x in r]for r in g]"
        
        return None
    
    def detect_arithmetic_pattern(self, task_data, analysis):
        """Detect arithmetic patterns"""
        ex = task_data['train'][0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        if inp.shape == out.shape:
            # Addition
            diff = out - inp
            if np.all(diff == diff[0, 0]):
                v = int(diff[0, 0])
                if v != 0:
                    return f"def p(g):\n return[[x+{v}for x in r]for r in g]"
            
            # Modulo
            for m in range(2, 10):
                if np.all(out == inp % m):
                    return f"def p(g):\n return[[x%{m}for x in r]for r in g]"
        
        return None
    
    def detect_object_move(self, task_data, analysis):
        """Detect object movement patterns"""
        # Complex pattern - placeholder
        return None
    
    def detect_fill_shape(self, task_data, analysis):
        """Detect shape filling patterns"""
        # Complex pattern - placeholder
        return None
    
    def detect_extract_largest(self, task_data, analysis):
        """Detect largest component extraction"""
        # Complex pattern - placeholder
        return None
    
    def detect_connect_dots(self, task_data, analysis):
        """Detect dot connection patterns"""
        # Complex pattern - placeholder
        return None
    
    def detect_symmetry_complete(self, task_data, analysis):
        """Detect symmetry completion"""
        # Complex pattern - placeholder
        return None
    
    def detect_pixel_art_scale(self, task_data, analysis):
        """Detect pixel art scaling"""
        # Already handled in regular scaling
        return None
    
    def try_combined_patterns(self, task_data, analysis):
        """Try combinations of patterns"""
        # Try rotate then crop
        for angle in [90, 180, 270]:
            rotated = self.apply_rotation(task_data['train'][0]['input'], angle)
            if self.matches_output(rotated, task_data['train'][0]['output']):
                return f"def p(g):\n r={'[list(r)for r in zip(*g[::-1])]' if angle==90 else '[r[::-1]for r in g[::-1]]' if angle==180 else '[list(r)for r in zip(*g)][::-1]'}\n return r"
        
        return None
    
    def apply_rotation(self, grid, angle):
        """Apply rotation to grid"""
        g = np.array(grid)
        if angle == 90:
            return np.rot90(g, -1).tolist()
        elif angle == 180:
            return np.rot90(g, 2).tolist()
        elif angle == 270:
            return np.rot90(g, -3).tolist()
        return grid
    
    def matches_output(self, grid1, grid2):
        """Check if two grids match"""
        return grid1 == grid2
    
    def print_pattern_statistics(self):
        """Print pattern usage statistics"""
        print("\n📊 Pattern Statistics:")
        print("="*60)
        
        sorted_patterns = sorted(self.pattern_stats.items(), 
                               key=lambda x: x[1]['successes'], 
                               reverse=True)
        
        for pattern, stats in sorted_patterns[:20]:
            if stats['attempts'] > 0:
                success_rate = stats['successes'] / stats['attempts'] * 100
                print(f"{pattern:30} | Attempts: {stats['attempts']:4} | "
                      f"Successes: {stats['successes']:4} | "
                      f"Rate: {success_rate:5.1f}%")

def main():
    """Main execution function"""
    print("🚀 Complete ARC Solver Starting...")
    print("="*60)
    
    # Determine environment
    in_kaggle = '/kaggle' in sys.path[0]
    
    if in_kaggle:
        base_path = "/kaggle/input/google-code-golf-2025"
        output_dir = "/kaggle/working/submission"
        zip_path = "/kaggle/working/submission.zip"
    else:
        base_path = "."
        output_dir = "submission"
        zip_path = "submission.zip"
    
    print(f"📁 Input path: {base_path}")
    print(f"📁 Output path: {output_dir}")
    print(f"📦 Zip path: {zip_path}")
    
    # Initialize solver
    solver = ComprehensiveARCSolver(enable_neural=False, verbose=True)
    solutions = {}
    successful = 0
    total_chars = 0
    
    # Process all tasks
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        
        try:
            # Load task
            task_path = f"{base_path}/task{task_id}.json"
            with open(task_path) as f:
                task_data = json.load(f)
            
            # Solve task
            solution = solver.solve_task(task_data, task_id)
            solutions[task_id] = solution
            
            # Verify solution
            if solver._verify_solution(solution, task_data):
                successful += 1
                total_chars += len(solution)
                print(f"\n✅ Task {task_id} solved! ({len(solution)} chars)")
            else:
                print(f"\n❌ Task {task_id} fallback used")
                
        except FileNotFoundError:
            print(f"\n⚠️  Task {task_id} - File not found: {task_path}")
            solutions[task_id] = "def p(g):\n return g"
        except Exception as e:
            print(f"\n⚠️  Task {task_id} - Error: {str(e)}")
            if solver.verbose:
                traceback.print_exc()
            solutions[task_id] = "def p(g):\n return g"
    
    # Print statistics
    solver.print_pattern_statistics()
    
    print("\n" + "="*60)
    print("📊 Final Statistics:")
    print(f"  Total tasks: 400")
    print(f"  Successful: {successful} ({successful/400*100:.1f}%)")
    print(f"  Average solution length: {total_chars/max(successful,1):.1f} chars")
    print(f"  Total characters: {total_chars}")
    
    # Create submission
    print("\n📦 Creating submission...")
    os.makedirs(output_dir, exist_ok=True)
    
    for task_id, code in solutions.items():
        with open(f"{output_dir}/task{task_id}.py", "w") as f:
            f.write(code)
    
    # Create zip file
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for task_id in solutions:
            zipf.write(f"{output_dir}/task{task_id}.py", f"task{task_id}.py")
    
    print(f"\n✨ Submission created: {zip_path}")
    print("🎉 Complete!")

if __name__ == "__main__":
    main()